# Environment Setup

In [1]:
# import required packages

# Install Pyomo optimization modeling package
!pip install pyomo
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

# Install optimization engine (solution algorithms)
# Linear program solver https://www.gnu.org/software/glpk/
!apt-get install -y -qq glpk-utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.2 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 123069 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.7.1+dfsg-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.7.1+dfsg-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.7.1+dfsg-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.7.1+dfsg-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.7.1+dfsg-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.7.1+dfsg-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-2_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-2) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack ...

## Optimization Model


In [5]:
# Initialize model by defining a pyomo instance
model = pyo.ConcreteModel()

# ------------------------------------
# Decision Variables                  |
# ------------------------------------

# r is the number of regular mix
# d is the number of deluxe mix
# h is the number of holidy mix
# s is the number of holidy mix
model.r = pyo.Var(bounds=(0, None))
model.d = pyo.Var(bounds=(0, None))
model.h = pyo.Var(bounds=(0, None))
model.s = pyo.Var(bounds=(0, None))

# Shortcut: redefine pyomo variable names to use shorthand
r = model.r
d = model.d
h = model.h
s = model.s

# ------------------------------------
# Objective Function                  |
# ------------------------------------

# Objective function definition
model.obj = pyo.Objective(expr = 1.65*r + 2.00*d + 2.25*h + 2.50*s, sense = maximize)

# ------------------------------------
# CONSTRAINTS                         |
# ------------------------------------

# Constraint definition, 8 constraints in total named C1, C2, C3, C4, C5, C6, C7, C8
model.C1 = pyo.Constraint(expr = 0.15*r + 0.2*d + 0.25*h + 0.50*s <= 6000, doc = 'Almond')
model.C2 = pyo.Constraint(expr = 0.25*r + 0.2*d + 0.15*h + 0.10*s <= 7500, doc = 'Brazil Nuts')
model.C3 = pyo.Constraint(expr = 0.25*r + 0.2*d + 0.15*h + 0.10*s <= 7500,  doc = 'Filberts')
model.C4 = pyo.Constraint(expr = 0.10*r + 0.2*d + 0.25*h + 0.10*s <= 6000,  doc = 'Pecans')
model.C5 = pyo.Constraint(expr = 0.25*r + 0.2*d + 0.20*h + 0.20*s <= 7500,  doc = 'Walnuts')
model.C6 = pyo.Constraint(expr = r >= 10000, doc = 'Regular Mix')
model.C7 = pyo.Constraint(expr = d >= 3000, doc = 'Deluxe Mix')
model.C8 = pyo.Constraint(expr = h >= 5000, doc = 'Holiday Mix')

# ------------------------------------
# SOLVE THE MODEL                     |
# ------------------------------------
# Specify the solver engine. Since this is a LP problem, we choose "glpk"
opt = SolverFactory('glpk')

# The following line generates sensitivity tables, but cannot read the
# variable names we have chosen (it creates generic names)
opt.options["ranges"] = 'sens.txt'

results = opt.solve(model)
model.pprint()

# ------------------------------------
# PRINT RESULTS                       |
# ------------------------------------
print("\n --------------- RESULTS --------------- \n");
print(str(results.solver))
print("Optimal objective function value =", model.obj());
print("Optimal number of Regular Mix to make =", r())
print("Optimal number of Deluxe Mix to make =", d())
print("Optimal number of Holiday Mix to make =", h())
print("Optimal number of Super Mix to make =", s())

4 Var Declarations
    d : Size=1, Index=None
        Key  : Lower : Value   : Upper : Fixed : Stale : Domain
        None :     0 : 10625.0 :  None : False : False :  Reals
    h : Size=1, Index=None
        Key  : Lower : Value  : Upper : Fixed : Stale : Domain
        None :     0 : 5000.0 :  None : False : False :  Reals
    r : Size=1, Index=None
        Key  : Lower : Value   : Upper : Fixed : Stale : Domain
        None :     0 : 17500.0 :  None : False : False :  Reals
    s : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 1.65*r + 2.0*d + 2.25*h + 2.5*s

8 Constraint Declarations
    C1 : Almond
        Size=1, Index=None, Active=True
        Key  : Lower : Body                            : Upper  : Active
        None :  -Inf : 0.15*r 

This is the mathemtically guaranteed optimal solution, meaning, there does not exist any other production plan (any other combination of $c$ and $p$) that can generate higher revenues while satisfying the constraints of the problem. Feel free to try it out below...

In [13]:
# How confident can we be that there is no other better solution out there?
# Can you find a feasible solution that can strictly beat the (4,25) solution in terms of revenues?
import pandas as pd
import numpy as np

#production plan
# Try out numbers here...
my_r=12000;
my_d=5000;
my_h=10000;
my_s=1000;

#revenues
my_obj = 1.65*my_r + 2.00*my_d + 2.25*my_h + 2.50*my_s

#constraints
my_c1 = 0.15*my_r + 0.2*my_d + 0.25*my_h + 0.50*my_s <= 6000
my_c2 = 0.25*my_r + 0.2*my_d + 0.15*my_h + 0.10*my_s <= 7500
my_c3 = 0.25*my_r + 0.2*my_d + 0.15*my_h + 0.10*my_s <= 7500
my_c4 = 0.10*my_r + 0.2*my_d + 0.25*my_h + 0.10*my_s <= 6000
my_c5 = 0.25*my_r + 0.2*my_d + 0.20*my_h + 0.20*my_s <= 7500
my_c6 = my_r >= 10000
my_c7 = my_d >= 3000
my_c8 = my_h >= 5000

my_results = {'pyomo_OFV':[model.obj()], 'my_OFV': [my_obj], 'my_r': [my_r], 'my_d': [my_d], 'my_h': [my_h], 'c1?': [my_c1], 'c2?': [my_c2], 'c3?': [my_c3], 'c4?': [my_c4], 'c5?': [my_c5], 'c6?': [my_c6], 'c7?': [my_c7], 'c8?': [my_c8]};
pd.DataFrame(data=my_results)

,pyomo_OFV,my_OFV,my_r,my_d,my_h,c1?,c2?,c3?,c4?,c5?,c6?,c7?,c8?
0,61375.0,54800.0,12000,5000,10000,True,True,True,True,True,True,True,True


## Sensitivity Analysis (part of the next lab)

In [7]:
# ------------------------------------
# SENSITIVITY ANALYSIS                |
# ------------------------------------
f = open('sens.txt', 'r')
file_contents = f.read()
print(file_contents)
f.close()

GLPK 4.65 - SENSITIVITY ANALYSIS REPORT                                                                         Page   1

Problem:    
Objective:  x1 = 61375 (MAXimum)

   No. Row name     St      Activity         Slack   Lower bound       Activity      Obj coef  Obj value at Limiting
                                          Marginal   Upper bound          range         range   break point variable
------ ------------ -- ------------- ------------- -------------  ------------- ------------- ------------- ------------
     1 c_u_x6_      NU    6000.00000        .               -Inf     5390.00000      -8.50000   56190.00000 c_l_x12_
                                           8.50000    6000.00000     6583.33333          +Inf   66333.33333 c_u_x9_

     2 c_u_x7_      BS    7250.00000     250.00000          -Inf     6500.00000      -1.50000   50500.00000 c_u_x10_
                                            .         7500.00000     7250.00000          +Inf          +Inf

     3 c_u_x8_  